In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox
import cv2
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
import math
import folium
import webbrowser
import os
import matplotlib.pyplot as plt
from PIL import Image, ImageTk
import threading
from flask import Flask, jsonify
from tensorflow.keras.models import load_model
from skimage.transform import resize

# Constants and tunable parameters
DILATION_KERNEL_SIZE = (15, 15)
DILATION_ITERATIONS = 1
MIN_LINE_LENGTH = 1
MAX_LINE_GAP = 2
SLOPE_THRESHOLD = 0.1
IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 3


class CropRowDetector:
    def __init__(self, model_path="C:\\Users\\lenovo\\Documents\\GitHub\\Mechatronics\\Codes\\crop_row_detection_model.h5"):
        self.model = load_model(model_path,compile=False)
        self.image_path = None
        self.cam_x = 0.0
        self.cam_y = 0.0
        self.cam_h = 0.0
        self.resolution = 0.0
        self.cam_angle = 0.0
        self.global_points = []
    
    def set_global_points(self, points):
        self.global_points = points
        
    def predict_mask(self, img):
        img = img[:, :, :IMG_CHANNELS]  # Ensure the image has 3 channels
        img_resized = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)  # Resize to U-Net input size
        img_resized = np.expand_dims(img_resized, axis=0)  # Add batch dimension
        img_resized = img_resized.astype(np.float32)  # Convert to float32
        
        prediction = self.model.predict(img_resized)[0]  # Predict mask
        predicted_mask = (prediction > 0.5).astype(np.uint8)  # Threshold to binary
        
        # Resize mask to original image size
        predicted_mask = resize(predicted_mask, (img.shape[0], img.shape[1]), mode='constant', preserve_range=True)
        
        # Ensure the mask is of type uint8 and single-channel
        predicted_mask = (predicted_mask * 255).astype(np.uint8)  # Scale to 0-255
        return predicted_mask

   

    def find_crop_rows(self, binary_mask):
        crop_rows = cv2.HoughLinesP(binary_mask, rho=1, theta=np.pi/180, threshold=50, minLineLength=MIN_LINE_LENGTH, maxLineGap=MAX_LINE_GAP)
        return crop_rows if crop_rows is not None else []

    def merge_lines(self, lines, eps=0.5, min_samples=2, slope_threshold=SLOPE_THRESHOLD):
        slopes = []
        intercepts = []
        for line in lines:
            for x1, y1, x2, y2 in line:
                if x2 != x1:
                    slope = (y2 - y1) / (x2 - x1)
                    intercept = y1 - slope * x1
                    slopes.append(slope)
                    intercepts.append(intercept)
        avg_slope = np.mean(slopes)
        filtered_lines = [(slope, intercept) for slope, intercept in zip(slopes, intercepts) if abs(slope - avg_slope) <= slope_threshold]
        features = np.array(filtered_lines)
        if len(features) == 0:
            return []
        scaler = StandardScaler()
        features_scaled = scaler.fit_transform(features)
        db = DBSCAN(eps=eps, min_samples=min_samples).fit(features_scaled)
        labels = db.labels_
        unique_labels = set(labels)
        merged_lines = []
        for label in unique_labels:
            if label == -1:
                continue
            cluster_lines = features[labels == label]
            avg_slope = np.mean(cluster_lines[:, 0])
            avg_intercept = np.mean(cluster_lines[:, 1])
            merged_lines.append((avg_slope, avg_intercept))
        return merged_lines

    def draw_lines(self, image, lines, color):
        for slope, intercept in lines:
            x1 = 0
            y1 = int(intercept)
            x2 = image.shape[1]
            y2 = int(slope * x2 + intercept)
            cv2.line(image, (x1, y1), (x2, y2), color, 5)

    def draw_middle_lines(self, image, merged_lines, middle_line_color=(255, 0, 0)):
        merged_lines.sort(key=lambda x: x[1])
        middle_lines = []
        for i in range(len(merged_lines) - 1):
            slope1, intercept1 = merged_lines[i]
            slope2, intercept2 = merged_lines[i + 1]
            avg_slope = (slope1 + slope2) / 2
            avg_intercept = (intercept1 + intercept2) / 2
            middle_lines.append((avg_slope, avg_intercept))
        self.draw_lines(image, middle_lines, middle_line_color)
        return middle_lines

    def draw_points_on_lines(self, image, lines, num_points=10, point_color=(0, 0, 255)):
        height, width, _ = image.shape
        points = []
        for slope, intercept in lines:
            x_values = np.linspace(0, width - 1, num_points)
            y_values = slope * x_values + intercept
            line_points = []
            for x, y in zip(x_values, y_values):
                x = int(x)
                y = int(y)
                if 0 <= x < width and 0 <= y < height:
                    cv2.circle(image, (x, y), 10, point_color, -1)
                    line_points.append((x, y))
            points.extend(line_points)
        return points

    def pixel_to_global(self, y_pixel, x_pixel, cam_x, cam_y, cam_h, resolution, cam_angle):
        meter_to_global = 0.00001
        pixel_size_meters = math.tan(cam_angle) * cam_h / (resolution / 2)
        y_centered = resolution / 2 - y_pixel
        x_centered = x_pixel - resolution / 2
        y_meters = y_centered * pixel_size_meters
        x_meters = x_centered * pixel_size_meters
        pixel_global_x = cam_x + y_meters * meter_to_global
        pixel_global_y = cam_y + x_meters * meter_to_global
        return pixel_global_x, pixel_global_y

    def process_image(self, image_path, cam_x, cam_y, cam_h, resolution, cam_angle):
        img = cv2.imread(image_path)
        if img is None:
            print(f"Error: Unable to read image at {image_path}")
            return None, None, None, None, None
    
        # Predict the binary mask using the U-Net model
        binary_mask = self.predict_mask(img)
    
        # Find crop rows based on the binary mask
        crop_rows = self.find_crop_rows(binary_mask)
    
        if crop_rows is None or len(crop_rows) == 0:
            print("No crop rows detected.")
            return img, None, binary_mask, None, None
    
        # Merge the detected crop rows
        merged_lines = self.merge_lines(crop_rows, eps=0.5, min_samples=2)
    
        # Draw the middle lines between merged crop rows
        middle_lines = self.draw_middle_lines(img, merged_lines) if merged_lines else []
    
        # Draw points along the middle lines
        points = self.draw_points_on_lines(img, middle_lines) if middle_lines else []
    
        # Convert pixel points to global coordinates
        global_points = [self.pixel_to_global(y, x, cam_x, cam_y, cam_h, resolution, cam_angle) for x, y in points]
    
        # Save processed images
        cv2.imwrite("processed_image_original.png", img)
        cv2.imwrite("processed_image_binary_mask.png", binary_mask * 255)
    
        if merged_lines:
            img_with_rows = img.copy()
            self.draw_lines(img_with_rows, merged_lines, (0, 0, 255))
            cv2.imwrite("processed_image_with_rows.png", img_with_rows)
    
            if middle_lines:
                img_with_middle_lines = img.copy()
                self.draw_lines(img_with_middle_lines, middle_lines, (255, 0, 0))
                cv2.imwrite("processed_image_with_middle_lines.png", img_with_middle_lines)
    
        return img, None, binary_mask, merged_lines, global_points



class CropRowDetectionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Crop Row Detection App")
        self.create_widgets()
        self.detector = CropRowDetector()
        self.cap = None
        self.frame = None
        self.stop_event = threading.Event()
        self.output_dir = "C:/Users/lenovo/Documents/Mechatronics/Source_Directory"
        self.flask_thread = threading.Thread(target=self.run_flask_server, daemon=True)
        self.flask_thread.start()
        
    def run_flask_server(self):
        app = Flask(__name__)
    
        @app.route('/data', methods=['GET'])
        def get_data():
            # Return the latest global points data
            return jsonify({"points": self.detector.global_points})
    
        app.run(host='0.0.0.0', port=5000)
        
    def create_widgets(self):
        self.label = tk.Label(self.root, text="Crop Row Detection App", font=("Helvetica", 16))
        self.label.pack(pady=10)
    
        self.choose_image_button = tk.Button(self.root, text="Choose Image", command=self.choose_image)
        self.choose_image_button.pack(pady=10)
    
        self.capture_video_button = tk.Button(self.root, text="Capture from Camera", command=self.start_video_capture)
        self.capture_video_button.pack(pady=10)
    
        self.stop_button = tk.Button(self.root, text="Stop Video Capture", command=self.stop_video_capture)
        self.stop_button.pack(pady=10)
    
        self.cam_x_label = tk.Label(self.root, text="Camera X Coordinate:")
        self.cam_x_label.pack()
        self.cam_x_entry = tk.Entry(self.root)
        self.cam_x_entry.pack()
    
        self.cam_y_label = tk.Label(self.root, text="Camera Y Coordinate:")
        self.cam_y_label.pack()
        self.cam_y_entry = tk.Entry(self.root)
        self.cam_y_entry.pack()
    
        self.cam_h_label = tk.Label(self.root, text="Camera Height (meters):")
        self.cam_h_label.pack()
        self.cam_h_entry = tk.Entry(self.root)
        self.cam_h_entry.pack()
    
        self.resolution_label = tk.Label(self.root, text="Camera Resolution:")
        self.resolution_label.pack()
        self.resolution_entry = tk.Entry(self.root)
        self.resolution_entry.pack()
    
        self.cam_angle_label = tk.Label(self.root, text="Camera Angle (degrees):")
        self.cam_angle_label.pack()
        self.cam_angle_entry = tk.Entry(self.root)
        self.cam_angle_entry.pack()
    
        self.process_button = tk.Button(self.root, text="Process Image", command=self.process_image)
        self.process_button.pack(pady=20)

    def choose_image(self):
        file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg;*.png;*.jpeg")])
        if file_path:
            self.detector.image_path = file_path
            print(f"Image chosen: {file_path}")

    def process_image(self):
        if not self.detector.image_path:
            messagebox.showerror("Error", "No image selected")
            return

        cam_x = float(self.cam_x_entry.get())
        cam_y = float(self.cam_y_entry.get())
        cam_h = float(self.cam_h_entry.get())
        resolution = float(self.resolution_entry.get())
        cam_angle = float(self.cam_angle_entry.get())

        img, _, binary_mask, merged_lines, global_points = self.detector.process_image(
            self.detector.image_path, cam_x, cam_y, cam_h, resolution, cam_angle)

        if img is None or binary_mask is None:
            messagebox.showerror("Error", "Failed to process the image")
            return

        # Display the processed images
        cv2.imshow("Processed Image", img)
        cv2.imshow("Binary Mask", binary_mask * 255)

        if merged_lines:
            img_with_lines = img.copy()
            self.detector.draw_lines(img_with_lines, merged_lines, (0, 255, 0))
            cv2.imshow("Processed Image with Lines", img_with_lines)

        if global_points:
            print("Global points:", global_points)
            self.detector.set_global_points(global_points)
            self.generate_map(global_points)

    def start_video_capture(self):
        self.stop_event.clear()
        self.cap = cv2.VideoCapture(0)
        if not self.cap.isOpened():
            print("Error: Unable to open camera")
            return

        self.capture_video()

    def capture_video(self):
        if self.stop_event.is_set():
            return
        ret, self.frame = self.cap.read()
        if ret:
            cv2.imshow("Video Capture", self.frame)
        self.root.after(10, self.capture_video)

    def stop_video_capture(self):
        self.stop_event.set()
        if self.cap:
            self.cap.release()
        cv2.destroyWindow("Video Capture")

    def generate_map(self, global_points):
        if not global_points:
            return

        map_ = folium.Map(location=[global_points[0][0], global_points[0][1]], zoom_start=15)
        for lat, lon in global_points:
            folium.Marker([lat, lon]).add_to(map_)

        map_path = os.path.join(self.output_dir, "map.html")
        map_.save(map_path)
        webbrowser.open(map_path)

if __name__ == "__main__":
    root = tk.Tk()
    app = CropRowDetectionApp(root)
    root.mainloop()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.7:5000
Press CTRL+C to quit


Image chosen: C:/Users/lenovo/Documents/Mechatronics/Source_Directory/5_2560_6144.jpg
1/1 [==============================] - 0s 145ms/step
Global points: [(1.0002719701217, 9.999515300773208), (1.0003662171935763, 9.999565565878209), (1.0004595666742917, 9.99961672857437), (1.0002540182984856, 9.999515300773208), (1.0003482653703617, 9.999565565878209), (1.0004416148510773, 9.99961672857437), (1.0002378616575924, 9.999515300773208), (1.000331211138308, 9.999565565878209), (1.0004254582101841, 9.99961672857437), (1.0002315785194673, 9.999515300773208), (1.0003285183648258, 9.999565565878209), (1.0004254582101841, 9.99961672857437), (1.0002181146520563, 9.999515300773208), (1.0003159520885756, 9.999565565878209), (1.0004137895250946, 9.99961672857437)]
1/1 [==============================] - 0s 32ms/step
Global points: [(1.0002719701217, 9.999515300773208), (1.0003662171935763, 9.999565565878209), (1.0004595666742917, 9.99961672857437), (1.0002540182984856, 9.999515300773208), (1.00034826